In [1]:
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import os
import pandas as pd

In [ ]:
import warnings
from doCorrelate import *
warnings.filterwarnings("ignore", category=RuntimeWarning)

omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/omni', x)), os.listdir('../Ordinary Solar Wind/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/artemis', x)), os.listdir('../Ordinary Solar Wind/artemis')))

file_pairs = [(os.path.join('../Ordinary Solar Wind/artemis', artemis_file), os.path.join('../Ordinary Solar Wind/omni', omni_file)) for artemis_file, omni_file in zip(artemisFileList, omniFileList) if not artemis_file.startswith('.') and not 'copy' in artemis_file]

for f in file_pairs:
    artemis_file = pd.read_csv(f[0], delimiter=',', header=0)
    omni_file = pd.read_csv(f[1], delimiter=',', header=0)
    artemis_file = artemis_file.rename(columns={'XPOS': 'Xpos'})
    # Reformat the time column to DateTime objects../Ordinary Solar Wind/omni'
    artemis_file['Time'] = pd.to_datetime(artemis_file['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_file['Time'] = pd.to_datetime(omni_file['Time'], format='%Y-%m-%d %H:%M:%S')
    artemis_file['V'] = np.sqrt(artemis_file['VX']**2 + artemis_file['VY']**2 + artemis_file['VZ']**2)
    omni_file['V'] = np.sqrt(omni_file['VX']**2 + omni_file['VY']**2 + omni_file['VZ']**2)

    print(artemis_file['Time'][0], omni_file['Time'][0])
    correlate((artemis_file, omni_file), 'outputs', 'V')

In [ ]:
from multiprocessing import Pool

from os import listdir
import csv

def process_file(file):
    #load the text file as list using csv module
    #run a bunch of operations
    #regex the int from the filename. for ex file1.txt returns 1, and file42.txt returns 42
    #write out a corresponsding csv file in dirB. For example input file file99.txt is written as out99.csv

if __name__ == '__main__':
    mypath = "some/path/"

    inputDir = mypath + 'dirA/'
    outputDir = mypath + 'dirB/'

    p = Pool(12)
    p.map(process_file, listdir(inputDir))

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
from concurrent.futures import ProcessPoolExecutor
from doCorrelate import correlate  # Ensure this is accessible

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Prepare file pairs
omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/omni', x)), os.listdir('../Ordinary Solar Wind/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/artemis', x)), os.listdir('../Ordinary Solar Wind/artemis')))

file_pairs = [
    (os.path.join('../Ordinary Solar Wind/artemis', artemis_file),
     os.path.join('../Ordinary Solar Wind/omni', omni_file))
    for artemis_file, omni_file in zip(artemisFileList, omniFileList)
    if not artemis_file.startswith('.') and 'copy' not in artemis_file
]

# Define the processing function
def process_files(file_pair, mode, base_output_dir):
    artemis_file = pd.read_csv(file_pair[0], delimiter=',', header=0)
    omni_file = pd.read_csv(file_pair[1], delimiter=',', header=0)
    artemis_file = artemis_file.rename(columns={'XPOS': 'Xpos'})
    artemis_file['Time'] = pd.to_datetime(artemis_file['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_file['Time'] = pd.to_datetime(omni_file['Time'], format='%Y-%m-%d %H:%M:%S')

    if mode == "Bx":
        output_dir = os.path.join(base_output_dir, 'Bx')
        os.makedirs(output_dir, exist_ok=True)
        print(f"Processing Bx: {artemis_file['Time'][0]} vs {omni_file['Time'][0]}")
        correlate((artemis_file, omni_file), output_dir, 'BX_GSM')
    elif mode == "V":
        output_dir = os.path.join(base_output_dir, 'V')
        os.makedirs(output_dir, exist_ok=True)
        artemis_file['V'] = np.sqrt(artemis_file['VX']**2 + artemis_file['VY']**2 + artemis_file['VZ']**2)
        omni_file['V'] = np.sqrt(omni_file['VX']**2 + omni_file['VY']**2 + omni_file['VZ']**2)
        print(f"Processing V (quadrature): {artemis_file['Time'][0]} vs {omni_file['Time'][0]}")
        correlate((artemis_file, omni_file), output_dir, 'V')

# Parallelize the tasks
output_dir_base = '../Ordinary Solar Wind/correlations'
with ProcessPoolExecutor() as executor:
    tasks = []
    for i, file_pair in enumerate(file_pairs[1:]):  # Skip the first pair if needed
        if i % 2 == 0:  # Task 1: Correlate Bx
            tasks.append(executor.submit(process_files, file_pair, "Bx", output_dir_base))
        else:  # Task 2: Correlate velocity magnitude V
            tasks.append(executor.submit(process_files, file_pair, "V", output_dir_base))

    # Wait for all tasks to complete
    for task in tasks:
        task.result()

In [ ]:
import os
import warnings
from concurrent.futures import ProcessPoolExecutor
from parallel_tasks_temp import process_files  # Import the function from the new module

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Prepare file pairs
omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/omni', x)), os.listdir('../Ordinary Solar Wind/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/artemis', x)), os.listdir('../Ordinary Solar Wind/artemis')))

file_pairs = [
    (os.path.join('../Ordinary Solar Wind/artemis', artemis_file),
     os.path.join('../Ordinary Solar Wind/omni', omni_file))
    for artemis_file, omni_file in zip(artemisFileList, omniFileList)
    if not artemis_file.startswith('.') and 'copy' not in artemis_file
]

# Parallelize the tasks
output_dir_base = '../Ordinary Solar Wind/correlations'
with ProcessPoolExecutor() as executor:
    tasks = []
    for i, file_pair in enumerate(file_pairs[1:]):  # Skip the first pair if needed
        if i % 2 == 0:  # Task 1: Correlate Bx
            tasks.append(executor.submit(process_files, file_pair, "Bx", output_dir_base))
        else:  # Task 2: Correlate velocity magnitude V
            tasks.append(executor.submit(process_files, file_pair, "V", output_dir_base))

    # Wait for all tasks to complete
    for task in tasks:
        task.result()

In [ ]:
import warnings
from doCorrelate import *
warnings.filterwarnings("ignore", category=RuntimeWarning)

omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('../Ordinary Solar Wind/omni'', x)), os.listdir('outputs/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('outputs/artemis', x)), os.listdir('outputs/artemis')))

file_pairs = [(os.path.join('outputs/artemis', artemis_file), os.path.join('outputs/omni', omni_file)) for artemis_file, omni_file in zip(artemisFileList, omniFileList) if not artemis_file.startswith('.') and not 'copy' in artemis_file]

for f in file_pairs:
    artemis_file = pd.read_csv(f[0], delimiter=',', header=0)
    omni_file = pd.read_csv(f[1], delimiter=',', header=0)
    artemis_file = artemis_file.rename(columns={'XPOS': 'Xpos'})
    # Reformat the time column to DateTime objects
    artemis_file['Time'] = pd.to_datetime(artemis_file['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_file['Time'] = pd.to_datetime(omni_file['Time'], format='%Y-%m-%d %H:%M:%S')

    print(artemis_file['Time'][0], omni_file['Time'][0])
    corr, shift = correlate_whole_day(artemis_file, omni_file)

In [ ]:
directory = '../Ordinary Solar Wind/correlations/Bz/metrics'
frame = []

# Sort filenames in date order
filenames = sorted(f for f in os.listdir(directory) if not f.startswith('.'))

# Append files in sorted order
for filename in filenames:
    file_path = os.path.join(directory, filename)
    file = pd.read_csv(file_path, delimiter=',', header=0, index_col=0)
    frame.append(file)

# Concatenate into a single DataFrame
df = pd.concat(frame, axis=0, ignore_index=True).reset_index(drop=True)

# Save to CSV
output_dir = '../Ordinary Solar Wind/correlations/Bz/merged'
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir, 'output.csv'))

In [ ]:
frame = []
directory = '../Ordinary Solar Wind/correlations/By/metrics'

for filename in os.listdir(directory):
    if not filename.startswith('.'):
        file_path = os.path.join(directory, filename)
        file = pd.read_csv(file_path, delimiter=',', header=0, index_col=0)
        frame.append(file)
df = pd.concat(frame, axis=0, ignore_index=True)
df = df.reset_index(drop=True)

if os.path.exists('../Ordinary Solar Wind/correlations/By/merged'):
    df.to_csv('../Ordinary Solar Wind/correlations/By/merged/output.csv')
else:
    os.makedirs('../Ordinary Solar Wind/correlations/By/merged')
    df.to_csv('../Ordinary Solar Wind/correlations/By/merged/output.csv')

In [ ]:
file = pd.read_csv('outputs/new-correlations-3/merged/output.csv', delimiter=',', header=0, index_col=0)

store = pd.DataFrame(columns=['Start', 'Stop', 'Pearson'])

filtered_df= file[(file['Pearson'] >= 0.65) & (file['Pearson'] <= 0.66)]
filtered_df.to_csv('5okdays/DaysAround065.csv')


filtered_df['Start'] = pd.to_datetime(filtered_df['Start']).dt.date
unique_date = filtered_df.drop_duplicates(subset=['Start'], keep='first')

#unique_date.to_csv('DatesAround85.csv', index=False)

In [ ]:
import os
import pandas as pd
omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('outputs/omni', x)), os.listdir('outputs/omni')))

minutes = []
for f in omniFileList:
    if not f.startswith('.'):
        omni_file = pd.read_csv(f'outputs/omni/{f}', delimiter=',', header=0)
        minutes.append(len(omni_file['Time']))

print(sum(minutes)-60)
print(sum(minutes)/60)